In [2]:
!pip install selenium

# 필요한 패키지 불러오기

In [3]:
from selenium import webdriver
import time
import pandas as pd
from bs4 import BeautifulSoup
import datetime
from datetime import timedelta
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup as bs
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
import pickle

# 호텔별 URL을 가져오기

In [4]:
# url 함수
def make_url(Url):
    import time
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome()
    driver.get(Url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    click1 = driver.find_element(By.XPATH,'//*[@id="__next"]/div/header/section/div/div[1]/section[2]/div/div/div/div/div[2]/div/div[1]/span[1]/span')
    click1.click()
    time.sleep(6)
    click2 = driver.find_element(By.XPATH,'/html/body/div[12]/div/div/section/div/div[3]/div[4]/section/div[2]/div/span[1]')
    click2.click()
    click3 = driver.find_element(By.XPATH,'/html/body/div[12]/div/div/section/div/div[3]/div[4]/section/div[2]/div/span[2]')
    click3.click()
    time.sleep(2)
    click4 = driver.find_element(By.XPATH,'/html/body/div[12]/div/div/footer/div/button[2]')
    click4.click()
    time.sleep(2)
    #무한 스크롤
    
    time.sleep(1)
    
    SCROLL_PAUSE_TIME = 1.5
    
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
    # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
        time.sleep(SCROLL_PAUSE_TIME)
        
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break
        
        last_height = new_height  

    soup = BeautifulSoup(html, 'html.parser')
    data = soup.select("section > div > div > a")
        
    for d in data:
        hotelurl.append(d.attrs['href'])
    # driver.close()
  

In [8]:
hotelurl = []
make_url("https://www.yanolja.com/hotel/r-900586?pageKey=1724956106096")

# 리뷰 수집

In [16]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome()

hotel = []
star = []
reviews = []
rev_sum_pos = []
rev_sum_neg = []
columns = ['hotel','star' ,'reviews', 'Review Summary Pos', 'Review Summary Neg','url' ]   


for url in hotelurl:
    cnt = 0
    SCROLL_PAUSE_TIME = 0.5
    time.sleep(1)
    Url = "https://www.yanolja.com"+url
    driver.get(Url)
    WebDriverWait(driver, 10)
    # try:
    # # 요소 찾기 시도
    try : 
        rev_sum_pos.append(driver.find_element(By.CLASS_NAME, 'css-4a3h28').text)
        element = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//div[text()='낮은 평점 요약']")))
        element.click()
        time.sleep(1)
    except NoSuchElementException:
        rev_sum_pos.append("")
    try:   
        element = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//div[text()='낮은 평점 요약']")))
        element.click()
        time.sleep(1)
    except TimeoutException:
        pass
    try :    
        rev_sum_neg.append(driver.find_element(By.CLASS_NAME, 'css-4a3h28').text) 
    except NoSuchElementException:
        rev_sum_neg.append("") 
    # except NoSuchElementException:
    # # 요소가 없을 경우 공백 문자열 설정
    #     rev_sum_pos = ''
    #     rev_sum_neg = ''
    
    hotel.append(driver.find_element(By.CSS_SELECTOR,'#__next > div > div > main > article > div.css-1cc3d9 > div > div.css-1e56347 > div.css-11vo59c > h1').text)
    
    # 리뷰 페이지 클릭 (Click review page)
    try:
        element2 = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME,'css-xa55ms'))
        )
    except NoSuchElementException :
        element2 = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME,'css-ufp1xj'))
        )
    except TimeoutException:
        element2 = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME,'css-ufp1xj'))
        )
    element2.click()
    # 무한 스크롤
    import time
    time.sleep(1)
    
    # SCROLL_PAUSE_TIME = 1.5
    element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CLASS_NAME, "css-stl2cn")))
    element.click()
    
    element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//div[text()='평점 낮은순']")))
    element.click()
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    max = int(driver.find_element(By.CLASS_NAME,"css-12rhouj").text.replace("(","").replace(")",""))
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
        time.sleep(SCROLL_PAUSE_TIME)
    
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if max > 1000:
            if(cnt>20):
                break
            cnt +=1
        if new_height == last_height:
            break
    
        last_height = new_height
    
    # 스크롤 끝나고 페이지 elements 가져오기
    star.append(float(driver.find_element(By.CLASS_NAME, 'css-u41ekg').text))
    soup = bs(driver.page_source, "lxml")
    
    # 특정 클래스 이름을 가진 모든 요소 찾기
    elements = soup.find_all('p')
    temp = ""
    # 데이터 추출 
    review = driver.find_elements(By.CLASS_NAME,"css-c92dc4")
    for i in review:
        temp += i.text
    reviews.append(temp)     
    
df = pd.DataFrame({"Hotel" : hotel, "Star" : star ,"Reviews" : reviews, "Review Pos" : rev_sum_pos,"Review Neg" : rev_sum_neg,"Url": hotelurl})   
df["Url"] ="https://www.yanolja.com"+df["Url"]

In [18]:
with open('ic.pickle', 'wb') as f:
    pickle.dump(df, f)